In [1]:
!pip install transformers datasets rouge-score nltk -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.4.0.6 whi

In [2]:
from datasets import load_dataset

# Load multi_news dataset
dataset = load_dataset("multi_news")

# Check keys and example
print(dataset)
print(dataset["train"][0])

# Extract each split into a variable
train_data = dataset["train"]
val_data = dataset["validation"]
test_data = dataset["test"]

# Optionally, convert to lists of texts and summaries
train_texts = train_data["document"]
train_summaries = train_data["summary"]

val_texts = val_data["document"]
val_summaries = val_data["summary"]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

multi_news.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


train.src.cleaned:   0%|          | 0.00/548M [00:00<?, ?B/s]

train.tgt:   0%|          | 0.00/58.8M [00:00<?, ?B/s]

...

test.tgt:   0%|          | 0.00/7.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})
{'document': 'National Archives \n \n Yes, it’s that time again, folks. It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. \n \n A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. \n \n Here at MarketBeat HQ, we’ll be offering

In [3]:
from transformers import AutoTokenizer

# Use appropriate tokenizer (e.g., for BART or T5)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

# Tokenize the dataset
def tokenize_fn(example):
    return tokenizer(example["document"], truncation=True, padding="max_length", max_length=1024)

def tokenize_target(example):
    return tokenizer(example["summary"], truncation=True, padding="max_length", max_length=128)

tokenized_train = train_data.map(lambda x: {**tokenize_fn(x), "labels": tokenize_target(x)["input_ids"]}, batched=True)
tokenized_val = val_data.map(lambda x: {**tokenize_fn(x), "labels": tokenize_target(x)["input_ids"]}, batched=True)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/44972 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForSeq2SeqLM

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

# Training arguments for just 1 epoch
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,  # Only one epoch
    predict_with_generate=True,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Trainer (no early stopping callback)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer
)

# Start training
trainer.train()


/tmp/ipykernel_35/50467039.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


<IPython.core.display.Javascript object>

In [ ]:
# Save model, tokenizer, and training arguments
trainer.save_model("./my_trained_model")
tokenizer.save_pretrained("./my_trained_model")
import shutil

# Zip the folder
shutil.make_archive("my_trained_model", 'zip', "./my_trained_model")


In [ ]:
from IPython.display import FileLink

# Create a download link
FileLink("my_trained_model.zip")


In [ ]:
# Evaluate on the validation set
metrics = trainer.evaluate()
print(metrics)


In [ ]:
# Predict on the test set
test_results = trainer.predict(test_dataset=tokenized_test)
print(test_results.metrics)
# Decode predictions
pred_summaries = tokenizer.batch_decode(
    test_results.predictions, skip_special_tokens=True
)

# Optionally view the first few summaries
for i in range(10):
    print(f"\nDocument: {test_data[i]['document'][:300]}...")
    print(f"Generated Summary: {pred_summaries[i]}")
    print(f"Reference Summary: {test_data[i]['summary']}")
